# Cleaning Data Programmatically

In this exercise, you will perform programtic cleaning of U.S. Bureau of Labor Statistics' "Occupational Employment and Wage Statistics (OEWS) Research Estimates by State and Industry". This dataset contains data on manager role occupations and the hourly wage. 

You will be cleaning the data for both data tidiness and data quality issues.

As a supplementary dataset, you are provided with an additional dataset, the 2021 1-year ACS PUMS dataset, to validate the data quality issues against.

In [1]:
#DO NOT MODIFY - imports
import pandas as pd
import numpy as np

## Datasets context

### OEWS data (uncleaned)

The OEWS dataset was gathered manually as a CSV from the U.S. Bureau of Labor Statistics' website. The data was narrowed down to specifically focus on the managerial domain.

The dataset has a number of variables - there are four variables of significance to us:

- AREA_TITLE: Area/location name, e.g. Alabama
- OCC_CODE: The Standard Occupational Classification (SOC) code, e.g. 11-0000
- OCC_TITLE: The Standard Occupational Classification (SOC) title, e.g. Management Occupations
- H_MEAN: The mean hourly wage of the worker, e.g. 61.13

**Legend**:
- `*` indicates that a wage estimate is not available
- `**` indicates an employement estimate is not available
- `#` indicates that a wage is equal to or greater than 100 dollars per hour or greater than 280,000 dollars per year
- `~` indicates a percent total less than 0.05%

### PUMS data (cleaned)

The PUMS dataset was downloaded via the Census Data API from the United Statest Census Bureau, and narrowed down for the Kern County - Bakersfield MSA, California area.

Dataset variables:

- WRK: Whether the individual worked last week.
    - 0: N/A (not reported)
    - 1: Worked
    - 2: Did not work
- SEX: Sex (Male / Female) of the individual
    - 1: Male
    - 2: Female 
- SCOP: Standard Occupational Classification (SOC) codes for 2018 and later, based on the 2018 SOC codes

In [8]:
#DO NOT MODIFY
#Read in the uncleaned excel file (note: will take a few minutes to load)
oews_data = pd.read_excel('oes_research_2021_sec_55-56.xlsx')
#Show the first few rows
oews_data.head()

,AREA,AREA_TITLE,NAICS,NAICS_TITLE,I_GROUP,OCC_CODE,OCC_TITLE,O_GROUP,TOT_EMP,EMP_PRSE,...,H_MEDIAN,H_PCT75,H_PCT90,A_PCT10,A_PCT25,A_MEDIAN,A_PCT75,A_PCT90,ANNUAL,HOURLY
0,1,Alabama,55,Management of Companies and Enterprises,sector,00-0000,All Occupations,total,21920,0,...,35.6,56.94,79.49,35470,47040,74050,118440,165330,NaN,NaN
1,1,Alabama,55,Management of Companies and Enterprises,sector,11-0000,Management Occupations,major,4820,4.1,...,61.13,92.03,#,61600,94020,127140,191420,#,NaN,NaN
2,1,Alabama,55,Management of Companies and Enterprises,sector,11-1021,General and Operations Managers,detailed,1600,7,...,60.5,#,#,60010,78520,125850,#,#,NaN,NaN
3,1,Alabama,55,Management of Companies and Enterprises,sector,11-2021,Marketing Managers,detailed,140,13.6,...,61.13,99.23,#,65240,98680,127140,206410,#,NaN,NaN
4,1,Alabama,55,Management of Companies and Enterprises,sector,11-2022,Sales Managers,detailed,140,14.7,...,49.56,77.94,#,59390,79010,103080,162110,#,NaN,NaN


In [6]:
#DO NOT MODIFY
#Read the cleaned .json file
cleaned_pums = pd.read_csv('cleaned_pums_2021.csv')
#Show the first few rows
cleaned_pums.head()

,WRK,SEX,SOCP
0,1,2,119151
1,2,1,119111
2,1,2,113121
3,1,1,1110XX
4,1,1,113051


## 1. Clean the Data Tidiness issues

### 1.1 Make a copy of the data

In order to ensure the raw dataframe is not impacted, make a copy of the OEWS data using the `df.copy()` functionality.

In [9]:
#FILL IN
#Make a copy of the OEWS data
cleaned_wage = oews_data.copy()

### 1.2 Clean the tidiness issue

For the Data Tidiness issue, let's look at the structure of the dataset using a combination of visual and programmatic assignment. 

From the OEWS wage data, we would like to keep the:

- `Area_Title`
- `OCC_CODE`
- `OCC_TITLE`
- `H_MEAN`

For `Area_Title`, keep only the data for **California**.

In [10]:
#FILL IN
#Filter the dataframe to the specific data elements
cleaned_wage = cleaned_wage[['AREA_TITLE', 'OCC_CODE', 'OCC_TITLE', 'H_MEAN']]
#Subset the dataframe for the Area_Title to only apply to California
cleaned_wage = cleaned_wage[cleaned_wage['AREA_TITLE'] == 'California']
#Describe the data
cleaned_wage.describe()

,AREA_TITLE,OCC_CODE,OCC_TITLE,H_MEAN
count,3223,3223,3223,3223.00
unique,1,436,436,1715.00
top,California,00-0000,All Occupations,18.89
freq,3223,22,22,13.00


### 1.3 Perform visual inspection of the dataset
Perform visual inspection of the dataset via the `.head()` function, particularly the dataframe's index, which is no longer in order. 

Reset the dataframe's index to fix this structural issue with the dataset - use the `reset_index()` with `drop` set to `True`.

In [11]:
# FILL IN - inspect the dataset
cleaned_wage.head()

,AREA_TITLE,OCC_CODE,OCC_TITLE,H_MEAN
377,California,00-0000,All Occupations,50.16
378,California,11-0000,Management Occupations,82.61
379,California,11-1011,Chief Executives,129.7
380,California,11-1021,General and Operations Managers,87.11
381,California,11-2011,Advertising and Promotions Managers,74.67


In [12]:
# FILL IN - reset the dataframe's index
cleaned_wage = cleaned_wage.reset_index(drop=True)
#Visually inspect the dataset again to confirm it worked
cleaned_wage.head()

,AREA_TITLE,OCC_CODE,OCC_TITLE,H_MEAN
0,California,00-0000,All Occupations,50.16
1,California,11-0000,Management Occupations,82.61
2,California,11-1011,Chief Executives,129.7
3,California,11-1021,General and Operations Managers,87.11
4,California,11-2011,Advertising and Promotions Managers,74.67


## 2. Clean the Data Quality issues

### 2.1 Completeness
There are many missing values and outliers in the wages data in the OEWS dataset, causing `H_MEAN` datatype to be object, instead of float.

- `*` indicates that a wage estimate is not available
- `#` indicates that a wage is equal to or greater than 100 dollars per hour or greater than 280,000 dollars per year

In this step, we will clean thess invalid values.

#### 2.1.1 Inspect the data types

In [13]:
#DO NOT MODIFY
#Briefly inspect the dtypes in the data
cleaned_wage.dtypes

AREA_TITLE    object
OCC_CODE      object
OCC_TITLE     object
H_MEAN        object
dtype: object

#### 2.1.2 Clean missing values and outliers
The invalid `*`  and `#` values cause `H_MEAN` datatype to be object, instead of float.

Let's remove the invalid data entries. First replace the missing values `*` and outliers `#` in `H_MEAN` with the `np.nan`. Then drop the NA values with the `dropna()` function. Finally, assert the number of NA values is 0.

In [14]:
#FILL IN to remove the rows with NA values

#1. Replace the * sign with np.nan for the H_MEAN data
#cleaned_wage['H_MEAN'] = ...
cleaned_wage['H_MEAN'] = cleaned_wage['H_MEAN'].replace({'*': np.nan})

#2. Deal with the outliers in the dataset
cleaned_wage['H_MEAN'] = cleaned_wage['H_MEAN'].replace({'#': np.nan})

#3. Drop the NA values
cleaned_wage = cleaned_wage.dropna()

#4. Assert the number of NA values is 0
assert cleaned_wage.isnull().sum().sum() == 0
cleaned_wage.head()

,AREA_TITLE,OCC_CODE,OCC_TITLE,H_MEAN
0,California,00-0000,All Occupations,50.16
1,California,11-0000,Management Occupations,82.61
2,California,11-1011,Chief Executives,129.70
3,California,11-1021,General and Operations Managers,87.11
4,California,11-2011,Advertising and Promotions Managers,74.67


#### 2.1.3 Inspect the cleaned data

After dropping the NAs, check the datatypes for the OEWS data variables programmatically via assert functions. `AREA_TITLE`, `OCC_CODE`, and `OCC_TITLE` should be of the `object` data type, and `H_MEAN` of the `float64` data type. 

In [15]:
#FILL IN
assert cleaned_wage.dtypes['AREA_TITLE'] == 'object'
assert cleaned_wage.dtypes['OCC_CODE'] == 'object'
assert cleaned_wage.dtypes['OCC_TITLE'] == 'object'
assert cleaned_wage.dtypes['H_MEAN'] == 'float64'

## 2.2 Consistency


There is an apparant lack of consistency between the OEWS and PUMS datasets with the occupation code variables, `SOCP` and `OCC_CODE`. The `OCC_CODE` variable in the OEWS dataset has hyphens as part of the code, whereas the PUMS' `SOCP` variable doesn't have this. In this step, we will address this issue.

#### 2.2.1 Add a OCC_CODE column to PUMS data

Create a new column within the PUMS dataframe called `OCC_CODE` that is a copy of the `SOCP` variable in the PUMS dataset. We'll be keeping the original `SOCP` column intact for potential cross-referencing.

In [17]:
#FILL IN
cleaned_pums['OCC_CODE'] = cleaned_pums['SOCP']
cleaned_pums.head()

,WRK,SEX,SOCP,OCC_CODE
0,1,2,119151,119151
1,2,1,119111,119111
2,1,2,113121,113121
3,1,1,1110XX,1110XX
4,1,1,113051,113051


#### 2.2.2 Clean the OCC_CODE in OEWS data

The `OCC_CODE` variable in the OEWS dataset has hyphens as part of the code, whereas the PUMS' `SOCP` variable doesn't have this. Modify the `OCC_CODE` variable within the `cleaned_wage` dataframe to remove the hyphen.

In [18]:
#FILL IN
#Enforce consistency between OCC_Code values between dataframes
#By replacing the hyphen in the OCC_Code variable
cleaned_wage['OCC_CODE'] = cleaned_wage['OCC_CODE'].replace('-', '', regex=True)
cleaned_wage.head()

,AREA_TITLE,OCC_CODE,OCC_TITLE,H_MEAN
0,California,000000,All Occupations,50.16
1,California,110000,Management Occupations,82.61
2,California,111011,Chief Executives,129.70
3,California,111021,General and Operations Managers,87.11
4,California,112011,Advertising and Promotions Managers,74.67


#### 2.2.3 Combine datasets 

We've cleaned the OEWS data. Now we can merge the `cleaned_pums` and `cleaned_wage` dataframes.

Drop the NA values and the unnecessary columns `Area_Title` which reports California for all values in the dataset. Drop the redundant `SOCP` variable. Finally reset the index after dropping the NA values.

*Note:* Here, we choose not to remove duplicate values - in the context of this problem statement, we value the individual counts to show the number of individuals in the sample belonging to a certain occupation (`OCC_TITLE`) / gender (`SEX`).

In [19]:
#DO NOT MODIFY
#Merge the two dataframes using only keys from the right frame
merged_df = pd.merge(cleaned_pums, cleaned_wage, on=['OCC_CODE'], how='right')
merged_df.head()

,WRK,SEX,SOCP,OCC_CODE,AREA_TITLE,OCC_TITLE,H_MEAN
0,NaN,NaN,NaN,000000,California,All Occupations,50.16
1,NaN,NaN,NaN,110000,California,Management Occupations,82.61
2,NaN,NaN,NaN,111011,California,Chief Executives,129.70
3,1.0,1.0,111021,111021,California,General and Operations Managers,87.11
4,1.0,1.0,111021,111021,California,General and Operations Managers,87.11


In [20]:
#FILL IN
#Drop NA
merged_df = merged_df.dropna()
#Drop the unnecessary columns `AREA_TITLE`, `SOCP`
merged_df = merged_df.drop(columns=['AREA_TITLE', 'SOCP'])
#Reset the index with drop=True setting
merged_df = merged_df.reset_index(drop=True)
merged_df.head()

,WRK,SEX,OCC_CODE,OCC_TITLE,H_MEAN
0,1.0,1.0,111021,General and Operations Managers,87.11
1,1.0,1.0,111021,General and Operations Managers,87.11
2,1.0,2.0,111021,General and Operations Managers,87.11
3,1.0,2.0,111021,General and Operations Managers,87.11
4,1.0,1.0,111021,General and Operations Managers,87.11


Now our final dataset contains information about occupational codes, titles, and the associated hourly wage of inviduals working in California in these postions, originating from the OEWS dataset. 

We also have the related gender of individuals working in these positions and whether or not they lasted last week coming in from the PUMS dataset, and we're ready for further analysis.

## 3. Store your data
Save your cleaned data to a csv, following best practices for storing and naming your cleaned data.

In [41]:
#FILL IN
merged_df.to_csv('pums_oews_2021_ocp_gender.csv', index=False)